<a href="https://colab.research.google.com/github/tutr464b/Decision-tree/blob/main/Decision_tree_training_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://cloudstore.zih.tu-dresden.de/index.php/s/gxBP63LNy9SgNFD/download -O adult.data.txt
!wget https://cloudstore.zih.tu-dresden.de/index.php/s/yw5ayzYk9g2bc3p/download -O adult.test.txt

--2021-12-08 16:02:13--  https://cloudstore.zih.tu-dresden.de/index.php/s/gxBP63LNy9SgNFD/download
Resolving cloudstore.zih.tu-dresden.de (cloudstore.zih.tu-dresden.de)... 141.30.62.64
Connecting to cloudstore.zih.tu-dresden.de (cloudstore.zih.tu-dresden.de)|141.30.62.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [text/plain]
Saving to: ‘adult.data.txt’

adult.data.txt      100%[===================>]   3.79M  --.-KB/s    in 0.1s    

2021-12-08 16:02:14 (26.4 MB/s) - ‘adult.data.txt’ saved [3974305/3974305]

--2021-12-08 16:02:14--  https://cloudstore.zih.tu-dresden.de/index.php/s/yw5ayzYk9g2bc3p/download
Resolving cloudstore.zih.tu-dresden.de (cloudstore.zih.tu-dresden.de)... 141.30.62.64
Connecting to cloudstore.zih.tu-dresden.de (cloudstore.zih.tu-dresden.de)|141.30.62.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2003153 (1.9M) [text/plain]
Saving to: ‘adult.test.txt’

adult.test.txt      100%[============


#data from TU Dresden


In [2]:
!ls

adult.data.txt	adult.test.txt	sample_data


In [3]:
import numpy as np
from math import log

In [4]:
def openfile(path, fname):
    """opens the file at path+fname and returns a list of examples and attribute values.
    examples are returned as a list with one entry per example. Each entry then
    is a list of attribute values, one of them being the class label. The returned list attr
    contains one entry per attribute. Each entry is a list of possible values or an empty list
    for numeric attributes.
    """
    datafile = open(path + fname, "r")
    examples = []
    for line in datafile:
        line = line.strip()
        line = line.strip('.')
        # ignore empty lines. comments are marked with a |
        if len(line) == 0 or line[0] == '|':
            continue
        ex = [x.strip() for x in line.split(",")]
        examples.append(ex)
    attr = []
    for i in range(len(examples[0])):
        values = list({x[i] for x in examples})  # set of all different attribute values
        if values[0].isdigit():  # if the first value is a digit, assume all are numeric
            attr.append([])
        else:
            attr.append(values)

    return examples, attr

In [5]:
def calc_entropy(examples, cls_index):
    """calculates the entropy over all examples. The index of the class label in the example
    is given by cls_index. Can also be the index to an attribute.
    """
    global attr
    # get attributes of examples with index cls_index
    example_classifications = [example[cls_index] for example in examples]
    # get unique counts of example_attributes
    unique, counts = np.unique(example_classifications, return_counts=True)

    # normalize counts to total number of examples for getting probs
    probs = counts/len(examples)
    # print(probs)

    entropy = -np.sum(probs*np.log2(probs))

    return entropy

In [6]:
def calc_ig(examples, attr_index, cls_index):
    """Calculates the information gain over all examples for a specific attribute. The
    class index must be specified.

    uses calc_entropy
    """
    global attr
    # get attributes of examples with index attr_indx
    example_attributes = [example[attr_index] for example in examples]

    # get unique counts of example_attributes
    unique, counts = np.unique(example_attributes, return_counts=True)

    # example split
    remainder = 0
    for j in range(len(unique)):
        # get all examples with attribute unique[j]
        examples_unique = [example for example in examples if example[attr_index] == unique[j]]
        remainder += len(examples_unique)/len(examples)*calc_entropy(examples_unique, cls_index)
    return calc_entropy(examples, cls_index) - remainder

In [7]:
def majority(examples, attr_index):
    """Returns the value of attribute "attr_index" that occurs the most often in the examples."""
    # create a flat list of all attribute values (with duplicates, so we can count)
    attr_vals = [example[attr_index] for example in examples]

    # among all unique attribute values, find the maximum with regards to occurrence in the attr_vals list

    maj_attr_val = max(set(attr_vals), key=attr_vals.count)
    return maj_attr_val

In [8]:

def choose_best_attr(examples, attr_avail, cls_index):
    """Iterates over all available attributes, calculates their information gain and returns the one
    that achieves the highest. attr_avail is a list of booleans corresponding to the list of attributes.
    it is true if the attribute has not been used in the tree yet (and is not numeric).
    """
    igs = []  # list of information gains for each attribute

    for j in range(len(attr)):
        if attr_avail[j]:
            igs.append(calc_ig(examples, j, cls_index))
        else:
            igs.append(-1)

    return igs.index(max(igs))  # return index of the attribute with highest IG

In [18]:
def dtree_learning(examples, attr_avail, default, cls_index):
  global attr
  if len(examples) == 0:
    return [None, default]
  elif len({x[cls_index] for x in examples}) == 1:
    #all the examples are classified in a class
    return [None, examples[0][cls_index]]
  elif attr_avail.count(True) == 0:
    #No more attribute to classify -> majority class prdiction
    return [None, majority(examples, cls_index)]
  else:
    best = choose_best_attr(examples, attr_avail, cls_index)
    tree = [best, []]
    for v in attr[best]:
        examples_v = [x for x in examples if x[best] == v]
        # why is it important to make a copy and not change it directly?
        new_attr_avail = attr_avail.copy()
        new_attr_avail[best] = False
        subtree = dtree_learning(examples_v, new_attr_avail, majority(examples, cls_index), cls_index)
        tree[1].append(subtree)
    return tree

In [11]:
def dtree_classify(dtree, x):
    """Classifies a single example x using the given decision tree. Returns the predicted class label.
    """
    # attribute index of splitting attribute
    attr_split_index = dtree[0]

    if attr_split_index is not None:
        # get attribute value for example
        example_split_attr = x[attr_split_index]

        # subtree position
        subtree_pos = attr[attr_split_index].index(example_split_attr)

        return dtree_classify(dtree[1][subtree_pos], x)  # descend into subtree recursively
    else:
        return dtree[1]

In [12]:
def dtree_test(dtree, examples, cls_index):
    """Classify all examples using the given decision tree. Prints the achieved accuracy."""
    correct = 0
    for j in range(len(examples)):
        if dtree_classify(dtree, examples[j]) == examples[j][cls_index]:
            correct += 1

    print("{} out of {} correct ({:.2f}%)".format(correct, len(examples), correct / len(examples) * 100))

In [14]:
###run the algorithm on real data

In [19]:
path = "./"  # directory of your data
datafile = "adult.data.txt"
testfile = "adult.test.txt"
examples, attr = openfile(path, datafile)  # load the training set
test, test_attr = openfile(path, testfile)  # load the test set
cls_index = len(attr) - 1  # the last attribute is assumed to be the class label
# attr_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "class"]

attr_avail = []  # marks which attributes are available for splitting (not numeric and not the class label)
for i in range(len(attr)):
    # the list attr[i] contains all possible values of attribute i. It is empty for numeric attributes.
    attr_avail.append(len(attr[i]) > 0 and i != cls_index)

# print(attr_avail)
dtree = dtree_learning(examples, attr_avail, [], cls_index)

print("Before removal of unknowns: ")
print("Training Set")
dtree_test(dtree, examples, cls_index)
print("Test Set")
dtree_test(dtree, test, cls_index)

# Extra task removal of unknowns
examples_removed = []
test_removed = []

for example in examples:
    # print(example.__contains__("?"))
    if not example.__contains__("?"):
        examples_removed.append(example)

for example in test:
    if not example.__contains__("?"):
        test_removed.append(example)

# print(len(examples_removed))
# print(len(test_removed))
dtree = dtree_learning(examples_removed, attr_avail, [], cls_index)

print("After removal of unknowns: ")
print("Training Set")
dtree_test(dtree, examples_removed, cls_index)
print("Test Set")
dtree_test(dtree, test_removed, cls_index)

Before removal of unknowns: 
Training Set
28186 out of 32561 correct (86.56%)
Test Set
13275 out of 16281 correct (81.54%)
After removal of unknowns: 
Training Set
25974 out of 30162 correct (86.11%)
Test Set
12203 out of 15060 correct (81.03%)
